In [1]:
# Установка необходимых модудей для работы кода
#! pip3 install pycodestyle flake8 pycodestyle_magic
#! pip3 install nltk
#! pip3 install pymorphy2
# %load_ext pycodestyle_magic
# %pycodestyle_on
# ! pip3 install wordcloud
# ! pip3 install pymystem3
# ! pip3 install pandas
# ! pip3 install seaborn
#! pip3 install gensim
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk import tokenize
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer() 
from pymystem3 import Mystem
m = Mystem()
import json
import pandas as pd
import collections
import os
import gensim
import urllib.request
from nltk.corpus import stopwords
from gensim.models import word2vec
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from nltk.corpus import stopwords
from nltk.probability import FreqDist

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1091)>


In [2]:
# Отзывы брались с сайта https://otzovik.com/reviews/kliningovaya_kompaniya_domovenok_russia_moscow/
# Пыталсь взять данные с сайта с помощью парсинга, однако после нескольких запусков кода сайт воспринимал мои запросы как запросы робота.
# Поэтому отзывы пришлось собирать вручную в файл с форматом .txt
# Ниже приводится код запроса парсингом:
# import requests
# from bs4 import BeautifulSoup

# URL = 'https://otzovik.com/reviews/kliningovaya_kompaniya_domovenok_russia_moscow/'
# HEADERS = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:71.0) Gecko/20100101 Firefox/71.0', 'accept': '*/*'}
# HOST = 'https://otzovik.com'


# def get_html(url, params=None):
#     r = requests.get(url, headers=HEADERS, params=params)
#     return r


# def get_pages(URL):
#     pages = []
#     html = get_html(URL)
#     if html.status_code == 200:
#         pages.append(URL)
#         soup = BeautifulSoup(html.text, 'html.parser')
#         items = soup.find_all('a', class_='pager-item nth')
#         for item in items:
#             pages.append(HOST+item['href'])
#     return pages


# def get_review_pages(pages):
#     review_pages = []
#     for page in pages:
#         html = get_html(page)
#         print(page)
#         if html.status_code == 200:
#             soup = BeautifulSoup(html.text, 'html.parser')
#             items = soup.find_all('a', class_='review-title')
#             for item in items:
#                 print(item)
#                 review_pages.append(item['href'])
#     return review_pages

# def parse():
#     pages = get_pages(URL)
#     review_pages = get_review_pages(pages)

# parse()

In [3]:
# Прочитаем  файл с плохими отзвами. 
with open('bad_comments.txt', encoding='utf-8') as file:
    text = file.read()
    text = text.lower()

In [4]:
# Приводим текст к нижнему регистру и токенизируем
words = [w.lower() for w in word_tokenize(text) if w.isalpha()]
analize_list = []
for token in words:
    token_analized = morph.parse(token)[0]
    analize_list.append(token_analized)
# Записываем text в analitics и затем записываем его в формат json
m = Mystem()
analitics = m.analyze(text)
with open("mystem.json","w", encoding="utf-8") as f:
    json.dump(analitics, f)

In [5]:
# Создадим датафрейм 
list_frame = []
lemmas = []
for token in analize_list:
    lemmas.append(token.normal_form)
    ch_dict = collections.OrderedDict()
    ch_dict["lex"]=token.normal_form
    ch_dict["word"]=token.word
    ch_dict["POS"]=token.tag.POS
    ch_dict["time"]=token.tag.tense
    ch_dict["gender"]=token.tag.gender
    ch_dict["case"]=token.tag.case
    ch_dict["verbal"]=token.tag.mood
    ch_dict["animacy"]=token.tag.animacy
    ch_dict["aspect"]=token.tag.aspect
    ch_dict["involvement"]=token.tag.involvement
    ch_dict["person"]=token.tag.person
    ch_dict["transitivity"]=token.tag.transitivity
    ch_dict["voice"]=token.tag.voice
    list_frame.append(ch_dict)
an_df = pd.DataFrame(list_frame).fillna('')
an_df.head(30)

,lex,word,POS,time,gender,case,verbal,animacy,aspect,involvement,person,transitivity,voice
0,не,не,PRCL,,,,,,,,,,
1,исполнять,исполняют,VERB,pres,,,indc,,impf,,3per,tran,
2,свой,свои,ADJF,,,accs,,inan,,,,,
3,обязанность,обязанности,NOUN,,femn,accs,,inan,,,,,
4,звонить,звонил,VERB,past,masc,,indc,,impf,,,intr,
5,примерно,примерно,ADVB,,,,,,,,,,
6,в,в,PREP,,,,,,,,,,
7,час,часов,NOUN,,masc,gent,,inan,,,,,
8,утро,утра,NOUN,,neut,gent,,inan,,,,,
9,год,года,NOUN,,masc,gent,,inan,,,,,


In [6]:
# Преобразуем текст в формат: только леммы, без знаков препинания, все в нижнем регистре,каждое предложение на отдельной строчке,
# без повтора слов. 
# Создадим новый файл, в который запишем полученные предложения.

with open('bad_cleaned.txt', 'w', encoding='utf-8') as file:
    sentences = tokenize.sent_tokenize(text)
    pattern1 = ' +'
    pattern2 = '[,.?!-/...]'
    for s in sentences:
        s = re.sub(pattern1, ' ', s)
        s = re.sub(pattern2, '', s)
        lemmas = m.lemmatize(s)
        lemmas = ''.join(lemmas)
        file.write(lemmas)

In [7]:
# Обучение модели
file = 'bad_cleaned.txt'
data = gensim.models.word2vec.LineSentence(file)
model = gensim.models.Word2Vec(data, vector_size=300, window=5, min_count=5, epochs=50)
model.init_sims(replace=True)
model_path = "bad_comments1"

print("Saving model...")
model.wv.save_word2vec_format(model_path, binary=True)

Saving model...


/Users/elizavetashu/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  """


In [8]:
vocab_len = len(model.wv)
vocab = vocab_len
print('Слов в модели:', vocab_len)

Слов в модели: 166


In [9]:
bad_words_list = list(model.wv.key_to_index)
print(sorted([w for w in bad_words_list]))

['10', '[персональный', 'а', 'без', 'больше', 'бригада', 'бы', 'быть', 'в', 'весь', 'во', 'вообще', 'вот', 'время', 'все', 'вы', 'вырезать', 'генеральный', 'говорить', 'год', 'грязный', 'данные', 'данный', 'день', 'деньги', 'для', 'до', 'домовенок', 'другой', 'если', 'естественно', 'еще', 'же', 'за', 'заказ', 'заказывать', 'заранее', 'звонить', 'звонок', 'знать', 'и', 'из', 'инвентарь', 'их', 'к', 'как', 'какой', 'качество', 'квартира', 'клиент', 'клинер', 'клининговый', 'когда', 'комната', 'компания', 'конечно', 'который', 'кухня', 'мебель', 'менеджер', 'месяц', 'минута', 'много', 'модератор]', 'мой', 'мочь', 'мы', 'мыть', 'на', 'надо', 'написать', 'не', 'некачественный', 'нет', 'никто', 'ничто', 'но', 'новый', 'нужно', 'нужный', 'о', 'обращаться', 'объяснять', 'один', 'окно', 'он', 'она', 'они', 'оператор', 'оставаться', 'от', 'отзыв', 'отменять', 'отношение', 'очень', 'первый', 'переделывать', 'по', 'позвонить', 'пол', 'полный', 'полтора', 'получать', 'получаться', 'пользоваться', '

In [10]:
# Прочитаем  файл с хорошими отзвами. 
with open('good_comments.txt', encoding='utf-8') as file:
    text = file.read()
    text = text.lower()

In [11]:
# Приводим текст к нижнему регистру и токенизируем
words = [w.lower() for w in word_tokenize(text) if w.isalpha()]
analize_list = []
for token in words:
    token_analized = morph.parse(token)[0]
    analize_list.append(token_analized)
# Записываем text в analitics и затем записываем его в формат json
m = Mystem()
analitics = m.analyze(text)
with open("mystem.json","w", encoding="utf-8") as f:
    json.dump(analitics, f)

In [12]:
# Создадим датафрейм 
list_frame = []
lemmas = []
for token in analize_list:
    lemmas.append(token.normal_form)
    ch_dict = collections.OrderedDict()
    ch_dict["lex"]=token.normal_form
    ch_dict["word"]=token.word
    ch_dict["POS"]=token.tag.POS
    ch_dict["time"]=token.tag.tense
    ch_dict["gender"]=token.tag.gender
    ch_dict["case"]=token.tag.case
    ch_dict["verbal"]=token.tag.mood
    ch_dict["animacy"]=token.tag.animacy
    ch_dict["aspect"]=token.tag.aspect
    ch_dict["involvement"]=token.tag.involvement
    ch_dict["person"]=token.tag.person
    ch_dict["transitivity"]=token.tag.transitivity
    ch_dict["voice"]=token.tag.voice
    list_frame.append(ch_dict)
an_df = pd.DataFrame(list_frame).fillna('')
an_df.head(30)

,lex,word,POS,time,gender,case,verbal,animacy,aspect,involvement,person,transitivity,voice
0,после,после,PREP,,,,,,,,,,
1,работа,работы,NOUN,,femn,gent,,inan,,,,,
2,компания,компании,NOUN,,femn,gent,,inan,,,,,
3,в,в,PREP,,,,,,,,,,
4,квартира,квартире,NOUN,,femn,loct,,inan,,,,,
5,быть,были,VERB,past,,,indc,,impf,,,intr,
6,очень,очень,ADVB,,,,,,,,,,
7,видный,видны,ADJS,,,,,,,,,,
8,изменение,изменения,NOUN,,neut,nomn,,inan,,,,,
9,в,в,PREP,,,,,,,,,,


In [13]:
# Преобразуем текст в формат: только леммы, без знаков препинания, все в нижнем регистре,каждое предложение на отдельной строчке, 
# без повтора слов. 
# Создадим новый файл, в который запишем полученные предложения.

with open('good_cleaned.txt', 'w', encoding='utf-8') as file:
    sentences = tokenize.sent_tokenize(text)
    pattern1 = ' +'
    pattern2 = '[,.?!-/...]'
    for s in sentences:
        s = re.sub(pattern1, ' ', s)
        s = re.sub(pattern2, '', s)
        lemmas = m.lemmatize(s)
        lemmas = ''.join(lemmas)
        file.write(lemmas)

In [14]:
# Обучение модели
file = 'good_cleaned.txt'
data = gensim.models.word2vec.LineSentence(file)
model = gensim.models.Word2Vec(data, vector_size=300, window=5, min_count=5, epochs=50)
model.init_sims(replace=True)
model_path = "good_comments1"

print("Saving model...")
model.wv.save_word2vec_format(model_path, binary=True)

Saving model...


/Users/elizavetashu/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  """


In [15]:
vocab_len = len(model.wv)
vocab = vocab_len
print('Слов в модели:', vocab_len)

Слов в модели: 97


In [16]:
good_words_list = list(model.wv.key_to_index)
print(sorted([w for w in good_words_list]))

['2', 'а', 'быстро', 'быть', 'в', 'вежливый', 'весь', 'время', 'все', 'всегда', 'год', 'график', 'даже', 'два', 'девушка', 'делать', 'день', 'для', 'довольный', 'домовенка', 'домовенок', 'другой', 'если', 'же', 'за', 'заказ', 'заказывать', 'знать', 'и', 'или', 'к', 'как', 'качественно', 'качество', 'квартира', 'клинер', 'когда', 'компания', 'который', 'кухня', 'много', 'мой', 'мочь', 'мы', 'на', 'не', 'неделя', 'нет', 'но', 'один', 'окно', 'она', 'они', 'оставаться', 'очень', 'по', 'поддерживать', 'пользоваться', 'понравиться', 'после', 'приезжать', 'приложение', 'приходить', 'проблема', 'просто', 'работа', 'раз', 'разный', 'рекомендовать', 'с', 'свой', 'сделать', 'себя', 'сервис', 'сотрудник', 'спасибо', 'так', 'такой', 'то', 'тот', 'у', 'убирать', 'убираться', 'уборка', 'удобный', 'уже', 'услуга', 'уходить', 'хороший', 'хорошо', 'цена', 'человек', 'чисто', 'что', 'чтобы', 'это', 'я']


In [17]:
# Посмотрим, какие слова могут повторяться в "ПЛОХИХ" и "ХОРОШИХ" отзывах
result = list(set(bad_words_list) & set(good_words_list))
print(result)

['уборка', 'так', 'время', 'раз', 'себя', 'они', 'качество', 'чтобы', 'уже', 'по', 'другой', 'к', 'а', 'компания', 'нет', 'мы', 'домовенок', 'что', 'как', 'за', 'с', 'заказ', 'человек', 'который', 'услуга', 'это', 'не', 'приезжать', 'же', 'у', 'мочь', 'мой', 'кухня', 'сервис', 'цена', 'день', 'то', 'заказывать', 'для', 'знать', 'год', 'клинер', 'и', 'весь', 'когда', 'пользоваться', 'на', 'в', 'я', 'оставаться', 'если', 'приходить', 'она', 'после', 'квартира', 'убирать', 'все', 'работа', 'быть', 'но', 'свой', 'много', 'такой', 'убираться', 'окно', 'просто', 'сотрудник', 'один', 'очень']


In [18]:
# Выберем слова, характерные только для "ПЛОХИХ" отзывов
result=list(set(bad_words_list) - set(good_words_list))
print(result)

['ничто', 'ремонт', 'еще', 'конечно', 'вы', 'сообщать', 'естественно', 'утро', 'клининговый', 'надо', 'сумма', 'звонить', 'при', 'во', 'присылать', 'пылесос', 'вообще', 'следующий', 'специалист', 'рабочий', 'получать', 'стена', 'переделывать', 'полный', 'модератор]', 'комната', 'звонок', 'минута', 'только', 'сказать', 'о', 'клиент', 'нужный', 'он', 'приходиться', 'связываться', 'грязный', 'без', 'постоянный', 'предупреждать', 'работник', 'никто', 'пыль', 'самый', 'деньги', 'получаться', 'через', 'час', 'их', 'заранее', 'хамство', 'написать', 'вот', 'менеджер', '[персональный', 'стоимость', 'строительный', 'новый', 'какой', 'оператор', 'отзыв', 'из', 'обращаться', 'средство', 'пол', 'данные', 'до', 'предлагать', 'смочь', 'рубль', 'позвонить', 'некачественный', 'бы', 'отношение', 'генеральный', 'этот', 'т', '10', 'работать', 'месяц', 'первый', 'бригада', 'мебель', 'мыть', 'объяснять', 'больше', 'полтора', 'инвентарь', 'нужно', 'сразу', 'от', 'сайт', 'со', 'говорить', 'данный', 'отменять'

In [19]:
# Выберем слова, характерные только для "ХОРОШИХ" отзывов
result=list(set(good_words_list) - set(bad_words_list))
print(result)

['график', 'качественно', 'хорошо', 'девушка', 'даже', 'хороший', 'приложение', 'чисто', 'домовенка', 'разный', 'рекомендовать', 'быстро', 'поддерживать', 'довольный', 'всегда', 'сделать', 'спасибо', 'два', '2', 'или', 'удобный', 'уходить', 'понравиться', 'тот', 'вежливый', 'проблема', 'неделя', 'делать']


In [20]:
# Как можно улучшить код:
# Вариант №1:
# Реализовать сбор данных напрямую с сайта с помощью парсинга.
# Начинать анализировать отзывы, начиная с количества звезд, "Достоинств" и "Недостатков".
# В случае, когда количество звезд от 1 до 2 и нет "Достоинств", программа должна заносить отзыв в категорию "ПЛОХОЙ".
# В случае, когда количество звезд от 4 до 5 и нет "Достоинств", программа должна заносить отзыв в категорию "ХОРОШИЙ".
# Исключить комментарии с количеством звезд 3, чтобы не портить статистику и не сбивать анализ.
# Собирать "ПЛОХИЕ" и "ХОРОШИЕ" отзывы в два отдельных файла.
#
#
# Вариант №2:
# В случае сбора данных из файла с форматом .txt анализировать "ПЛОХИЕ" и "ХОРОШИЕ" отзывы из одного файла,
# а не из двух разных файлов с "ПЛОХИМИ" и "ХОРОШИМИ" отзывами.
# Создать цикл, который будет обрабатывать слова из файла с форматом .txt, откинет повторяющиеся слова, оставит только уникальные,
# запишет их в два словаря:
# Один со словами, характерными для "ХОРОШИХ" отзывов;
# Второй со словами, характерными для "ПЛОХИХ" отзывов.
